# 2D indentation using Argiope & Hardness



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!type local_settings.py

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import argiope as ag
import hardness as hd
import pandas as pd
import numpy as np
import os, subprocess, time, local_settings, time, sys
%matplotlib nbagg

mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['contour.negative_linestyle'] = 'solid'

# USEFUL FUNCTIONS
def create_dir(path):
  try:
    os.mkdir(path)
  except:
    pass

## Settings

In [ ]:
# SETTINGS
workdir   = "_workdir/"
outputdir = "outputs/"
label   = "indentation_2D"

create_dir(workdir)
create_dir(workdir + outputdir)    
print (local_settings.ABAQUS_PATH)

## Model definition

In [ ]:
#-------------------------------------------------------------------------------
# MESH DEFINITIONS
def element_map(mesh):
    mesh.elements.loc[mesh.elements.type.argiope == "tri3", ("type", "solver", "")] = "CAX3" 
    mesh.elements.loc[mesh.elements.type.argiope == "quad4", ("type", "solver", "")] = "CAX4R" 
    return mesh
    
def sample_material_map(mesh):
    mesh.elements["materials"] = "SAMPLE_MAT" 
    return mesh

def indenter_material_map(mesh):
    mesh.elements["materials"] = "INDENTER_MAT" 
    return mesh
    
    
parts = {
    "sample" : hd.models.Sample2D(lx = 1., ly = 1., 
                                   r1 = 2., r2 = 3., 
                                   Nx = 32, Ny = 16,
                                   Nr = 2, Nt = 64, 
                                   gmsh_path = "gmsh",
                                   file_name = "dummy", 
                                   workdir = workdir, 
                                   gmsh_space = 2, 
                                   gmsh_options = "-algo delquad",
                                   element_map = element_map,
                                   material_map = sample_material_map),
                                   
    "indenter" : hd.models.SpheroconicalIndenter2D(
                                   R = 0.2,
                                   psi= 70.3, 
                                   r1 = 1., 
                                   r2 = 3., 
                                   r3 = 4., 
                                   lc1 = .1, 
                                   lc2 = .5,
                                   rigid = False,
                                   gmsh_path = "gmsh",
                                   file_name = "dummy", 
                                   workdir = workdir, 
                                   gmsh_space = 2, 
                                   gmsh_options = "-algo delquad",
                                   element_map = element_map,
                                   material_map = indenter_material_map)}
                                   
materials = [ag.materials.ElasticPlasticRateDep(label = "SAMPLE_MAT", young_modulus = 100000.0, poisson_ratio = 0.3,  
                                   yield_stress = 10.0, multiplier = 1.0e-6, exponent = 1.4),
             ag.materials.Elastic(label = "INDENTER_MAT", young_modulus = 1000000.0, poisson_ratio = 0.07)]


#-------------------------------------------------------------------------------
# AMPLITUDE  (load vs time law)

amplitude = ag.models.Amplitude(type = "Exp",
                     name = "Exp100",
                     duration = 100,
                     Nb_data_points = 100, 
                     load0 = 10, 
                     strain_rate0 = .01)

#-------------------------------------------------------------------------------
# STEP DEFINTIONS
steps = [hd.models.Step2D(name = "LOADING1",
                         control_type = "disp", 
                         duration = 1.,
                         kind = "adaptative",  
                         nframes = 50,
                         controlled_value = -0.02,
                         field_output_frequency = 99999),
hd.models.Step2D(name = "LOADING2",
                         control_type = "force", 
                         duration = 1.,
                         kind = "adaptative",  
                         nframes = 50,
                         controlled_value = -800.0,
                         field_output_frequency = 99999),
hd.models.Step2D(name = "LOADING3",
                         control_type = "amplitude",   
                         duration = amplitude.duration,
                         kind = "adaptative",  
                         nframes = 50,
                         controlled_value = -80.0,
                         amplitude = amplitude,
                         field_output_frequency = 99999),
hd.models.Step2D(name = "LOADING4",
                         control_type = "force", 
                         duration = 1.,
                         kind = "adaptative",  
                         nframes = 50,
                         controlled_value = 0.0,
                         field_output_frequency = 99999),
]                                                                                                  

model0 = hd.models.Indentation2D(label = label, 
                      parts = parts, 
                      steps = steps,
                      materials = materials, 
                      amplitude = amplitude, 
                      solver = "abaqus", 
                      solver_path = local_settings.ABAQUS_PATH,
                      workdir = workdir,
                      verbose = True)

In [ ]:
print("1: Preprocessing ----------------------------------")
%time model0.write_input()

In [ ]:
print("2: Processing -------------------------------------")
%time model0.run_simulation()

In [ ]:
print("3: Postprocessing ---------------------------------")
%time model0.postproc()

In [ ]:
print("4: Saving model -----------------------------------")
%time model0.save(workdir + "model.pcklz")
model = ag.utils.load(workdir + "model.pcklz")

## Model checking

Mesh building and quality checking.

In [ ]:
parts["indenter"].mesh.elements.head()

In [ ]:
parts["sample"].mesh.elements.head()

In [ ]:
i = 1
fig = plt.figure()
parts_names = parts.keys()
for name, part in parts.items(): 
    mesh = part.mesh
    patches = mesh.to_polycollection(edgecolor = "black", linewidth = .5, alpha = 1.)
    stats = mesh.stats()
    patches.set_array( stats.stats.max_abs_angular_deviation )
    patches.set_cmap(mpl.cm.YlOrRd)
    ax = fig.add_subplot(1, 2, i)
    ax.set_aspect("equal")
    ax.set_xlim(mesh.nodes.coords.x.min(), mesh.nodes.coords.x.max())
    ax.set_ylim(mesh.nodes.coords.y.min(), mesh.nodes.coords.y.max())
    ax.add_collection(patches)
    cbar = plt.colorbar(patches, orientation = "horizontal")
    cbar.set_label("Max Abs. Angular Deviation [$^o$]")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.grid()
    plt.title(name.title())
    i+= 1
plt.show()


## Simulation

## Post-Processing

### Time data


In [ ]:
hist = model.data["history"]
hist.head()

In [ ]:

plt.figure(figsize=plt.figaspect(0.2))
ax1 = plt.subplot(1,3,1)

for step, group in hist.groupby("step"):
  ax1.plot(-group.dtot, -group.F, label = "Step {0}".format(step))
ax1.grid()
ax1.legend(loc = "best")
ax1.set_ylabel("Total force $F$, [N]")
ax1.set_xlabel("Displacement, $\delta$ [mm]")

ax1=plt.subplot(1,3,2)
for step, group in hist.groupby("step"):
  ax1.plot(group.t, -group.F, label = "Step {0}".format(step))
ax1.grid()
ax1.legend(loc = "best")
ax1.set_ylabel("Total force $F$, [N]")
ax1.set_xlabel("Time [s]")


ax1=plt.subplot(1,3,3)
for step, group in hist.groupby("step"):
  ax1.plot(group.t, -group.dtot, label = "Step {0}".format(step))
ax1.grid()
ax1.legend(loc = "best")
ax1.set_ylabel("Displacement, $\delta$ [mm]")
ax1.set_xlabel("Time [s]")

plt.show()


### Fields

In [ ]:
model.parts["sample"].mesh.fields_metadata()

In [ ]:
model.parts["sample"].mesh.fields_metadata()

In [ ]:
parts = {k:part.mesh.copy() for k, part in model.parts.items() }

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect("equal")
ax.set_xlim(0., 4.)
ax.set_ylim(-2., 2.)

field_num = 14 
disp_num = 15
levels = np.linspace(-1.e-2, 1.e-2, 11)

for k, mesh in parts.items():
    field = mesh.fields[field_num].data.v12
    disp = mesh.fields[disp_num].data
    mesh.nodes[("coords", "x")] += disp.v1
    mesh.nodes[("coords", "y")] += disp.v2
    tri = mesh.to_triangulation()
    patches = mesh.to_polycollection(facecolor = "none",
                                     edgecolor = "black",
                                     linewidth = .5) 
    
    grad = ax.tricontourf(tri, field, levels, cmap = mpl.cm.jet, alpha = 1.)
    ax.tricontour(tri, field, levels, colors = "white", linewidths = 1.)
    ax.add_collection(patches)
cbar = plt.colorbar(grad)
cbar.set_label("Cauchy Stress, $\sigma_{12}$")
ax.axis("off")
plt.xlabel("$x$")
plt.ylabel("$y$")
#plt.grid()